ConvLSTM trained on simulated streamflow.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn, utils
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate, conv_lstm, datasets
import torch.autograd as autograd
import pickle

time_stamp = datetime.now().strftime('%Y%m%d-%H%M%S')
time_stamp

'20190812-210928'

In [2]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../log.out', mode='a')
chandler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - {} - %(message)s'.format(time_stamp))
fhandler.setFormatter(formatter)
chandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.addHandler(chandler)
logger.setLevel(logging.INFO)

In [3]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
num_devices = torch.cuda.device_count() if USE_CUDA else 0
logger.warning('cuda devices: {}'.format(list(torch.cuda.get_device_name(i) for i in range(num_devices))))
torch.manual_seed(0)
np.random.seed(0)

2019-08-12 21:09:28,425 - 20190812-210928 - cuda devices: []


In [4]:
landcover_nc = nc.Dataset('../data/NA_NALCMS_LC_30m_LAEA_mmu12_urb05_n40-45w75-90_erie.nc', 'r')
landcover_nc.set_auto_mask(False)
erie_lats = landcover_nc['lat'][:][::-1]
erie_lons = landcover_nc['lon'][:]
landcover_nc.close()
erie_lat_min, erie_lat_max, erie_lon_min, erie_lon_max = erie_lats.min(), erie_lats.max(), erie_lons.min(), erie_lons.max()
del erie_lats, erie_lons

out_lats, out_lons = load_data.load_dem_lats_lons()
out_lats = out_lats[(erie_lat_min <= out_lats) & (out_lats <= erie_lat_max)].copy()
out_lons = out_lons[(erie_lon_min <= out_lons) &  (out_lons <= erie_lon_max)].copy()

In [5]:
seq_len = 8
seq_steps = 1
stateful_lstm = False
validation_fraction, val_start, val_end = None, None, None

if stateful_lstm:
    val_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len * seq_steps)  # first day for which to make a prediction in train set
    val_end = '2010-09-30'
    train_start = '2010-10-01'
    train_end = '2012-12-31'
else:
    validation_fraction = 0.1
    train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len * seq_steps)  # first day for which to make a prediction in train set
    train_end = '2012-12-31'
test_start = '2013-01-01'
test_end = '2014-12-31'

In [18]:
rdrs_vars = [4, 5]
agg = ['sum', 'minmax']
include_month = True
train_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, train_start, train_end, aggregate_daily=agg, include_months=True, include_simulated_streamflow=True, resample_rdrs=True, out_lats=out_lats, out_lons=out_lons)
if stateful_lstm:
    val_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, val_start, val_end, conv_scalers=train_dataset.conv_scalers, aggregate_daily=agg,  include_months=True, include_simulated_streamflow=True, resample_rdrs=True, out_lats=out_lats, out_lons=out_lons)
test_dataset = datasets.RdrsGridDataset(rdrs_vars, seq_len, seq_steps, test_start, test_end, conv_scalers=train_dataset.conv_scalers, aggregate_daily=agg, include_months=True, include_simulated_streamflow=True, resample_rdrs=True, out_lats=out_lats, out_lons=out_lons)

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3296: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
landcover_types = []
geophysical_dataset = datasets.GeophysicalGridDataset(dem=True, landcover=False, soil=False, groundwater=False, min_lat=erie_lat_min, max_lat=erie_lat_max, min_lon=erie_lon_min, max_lon=erie_lon_max, landcover_types=landcover_types)
geophysical_data = next(geophysical_dataset.__iter__())

In [20]:
subbasins = train_dataset.simulated_streamflow['subbasin'].unique()
np.random.seed(0)
test_subbasins = np.random.choice(subbasins, size=int(0.2 * len(subbasins)), replace=False)
val_subbasins = np.random.choice(list(s for s in subbasins if s not in test_subbasins), size=int(validation_fraction * len(subbasins)), replace=False)
train_subbasins = list(s for s in subbasins if s not in test_subbasins and s not in val_subbasins)
station_subbasins = train_dataset.simulated_streamflow[~pd.isna(train_dataset.simulated_streamflow['StationID'])]['subbasin'].unique()

train_subbasin_indices = list(train_dataset.outlet_to_row_col[s] for s in train_subbasins)
val_subbasin_indices = list(train_dataset.outlet_to_row_col[s] for s in val_subbasins)
test_subbasin_indices = list(test_dataset.outlet_to_row_col[s] for s in test_subbasins)

train_mask = torch.zeros((train_dataset.out_lats.shape[0], train_dataset.out_lats.shape[1]), dtype=torch.bool)
val_mask = torch.zeros((train_dataset.out_lats.shape[0], train_dataset.out_lats.shape[1]), dtype=torch.bool)
for row in range(train_mask.shape[0]):
    for col in range(train_mask.shape[1]):
        train_mask[row, col] = True if (row, col) in train_subbasin_indices else False
        val_mask[row, col] = True if (row, col) in val_subbasin_indices else False
train_mask = train_mask.reshape(-1).to(device)
val_mask = val_mask.reshape(-1).to(device)

In [21]:
# Train model
num_epochs = 300
learning_rate = 2e-3
patience = 100
min_improvement = 0.01
best_loss_model = (-1, np.inf, None)

# Prepare model
batch_size = 32
num_convlstm_layers = 2
num_conv_layers = 6
convlstm_hidden_dims = [8] * num_convlstm_layers
conv_hidden_dims = [8] * (num_conv_layers - 1)
convlstm_kernel_size = [(5,5)] * num_convlstm_layers
conv_kernel_size = [(5,5)] * num_conv_layers
conv_activation = nn.LeakyReLU
dropout = 0.1
weight_decay = 1e-6

model = conv_lstm.ConvLSTMGridWithGeophysicalInput((train_dataset.conv_height, train_dataset.conv_width), train_dataset.n_conv_vars, 
                                                   geophysical_dataset.shape[0], convlstm_hidden_dims, conv_hidden_dims, convlstm_kernel_size, 
                                                   conv_kernel_size, num_convlstm_layers, num_conv_layers, conv_activation, dropout=dropout, 
                                                   geophysical_size=geophysical_dataset.shape[1:]).to(device)
if num_devices > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(num_devices)))
loss_fn = evaluate.NSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

writer = SummaryWriter(comment='ConvLSTM_simulationTraining')
param_description = {'time_stamp': time_stamp, 'H_convlstm': convlstm_hidden_dims, 'H_conv': conv_hidden_dims, 'batch_size': batch_size, 'num_convlstm_layers': num_convlstm_layers, 'num_conv_layers': num_conv_layers, 'convlstm_kernel_size': convlstm_kernel_size, 'conv_kernel_size': conv_kernel_size, 'loss': loss_fn, 
                     'optimizer': optimizer, 'lr': learning_rate, 'patience': patience, 'min_improvement': min_improvement, 'stateful_lstm': stateful_lstm, 'dropout': dropout, 'geophys_shape': geophysical_dataset.shape, 'conv_activation': conv_activation,
                     'num_epochs': num_epochs, 'seq_len': seq_len, 'seq_steps': seq_steps, 'train_start': train_start, 'train_end': train_end, 'weight_decay': weight_decay, 'validation_fraction': validation_fraction, 'landcover_types': landcover_types,
                     'test_start': test_start, 'test_end': test_end, 'n_conv_vars': train_dataset.n_conv_vars, 'model': str(model).replace('\n','').replace(' ', ''), 'val_start': val_start, 'val_end': val_end,
                     'train len': len(train_dataset), 'conv_height': train_dataset.conv_height, 'conv_width': train_dataset.conv_width, 'test len': len(test_dataset)}
writer.add_text('Parameter Description', str(param_description))
str(param_description)

"{'time_stamp': '20190812-210928', 'H_convlstm': [8, 8], 'H_conv': [8, 8, 8], 'batch_size': 32, 'num_convlstm_layers': 2, 'num_conv_layers': 4, 'convlstm_kernel_size': [(5, 5), (5, 5)], 'conv_kernel_size': [(5, 5), (5, 5), (5, 5), (5, 5)], 'loss': NSELoss(), 'optimizer': Adam (\nParameter Group 0\n    amsgrad: False\n    betas: (0.9, 0.999)\n    eps: 1e-08\n    lr: 0.002\n    weight_decay: 1e-05\n), 'lr': 0.002, 'patience': 100, 'min_improvement': 0.01, 'stateful_lstm': False, 'dropout': 0.2, 'geophys_shape': torch.Size([1, 460, 848]), 'conv_activation': <class 'torch.nn.modules.activation.LeakyReLU'>, 'num_epochs': 300, 'seq_len': 8, 'seq_steps': 1, 'train_start': datetime.datetime(2010, 1, 9, 0, 0), 'train_end': '2012-12-31', 'weight_decay': 1e-05, 'validation_fraction': 0.1, 'landcover_types': [], 'test_start': '2013-01-01', 'test_end': '2014-12-31', 'n_conv_vars': 15, 'model': 'ConvLSTMGridWithGeophysicalInput((conv_lstm):ConvLSTM((cell_list):ModuleList((0):ConvLSTMCell((conv):Conv

In [22]:
if stateful_lstm:
    train_sampler = datasets.StatefulBatchSampler(train_dataset, batch_size)
    val_sampler = datasets.StatefulBatchSampler(val_dataset, batch_size)
    test_sampler = datasets.StatefulBatchSampler(test_dataset, batch_size)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_sampler, pin_memory=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_sampler=val_sampler, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_sampler=test_sampler, pin_memory=True)
else:
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True, pin_memory=True, drop_last=False)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size, shuffle=False, pin_memory=True, drop_last=False)
    
geophysical_batch = geophysical_data.repeat(batch_size,1,1,1).to(device, non_blocking=True)

In [23]:
torch.manual_seed(0)
np.random.seed(0)
for epoch in range(num_epochs):
    model.train()

    train_losses = torch.tensor(0.0)
    val_losses = torch.tensor(0.0)
    conv_hidden_states = None
    for i, train_batch in enumerate(train_dataloader):
        y_train = train_batch['y_sim'].reshape((train_batch['y_sim'].shape[0],-1)).to(device, non_blocking=True)
        geophysical_input = geophysical_batch[:y_train.shape[0]]
        
        if not train_mask.any():
            print('Batch {} has no target values. skipping.'.format(i))
            continue
        if not stateful_lstm:
            conv_hidden_states = None
        
        y_pred, conv_hidden_states = model(train_batch['x_conv'].to(device), geophysical_input, hidden_state=conv_hidden_states)
        y_pred = y_pred.reshape((train_batch['y_sim'].shape[0], -1))
        train_loss = loss_fn(y_pred[:,train_mask], y_train[:,train_mask])
        val_losses += loss_fn(y_pred[:,val_mask], y_train[:,val_mask]).detach()

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        train_losses += train_loss.detach()
        
    train_loss = (train_losses / len(train_dataloader)).item()
    val_loss = (val_losses / len(train_dataloader)).item()
    print('Epoch', epoch, 'mean train loss:\t{}'.format(train_loss))
    print('Epoch', epoch, 'mean val loss:\t{}'.format(val_loss))
    writer.add_scalar('loss_nse', train_loss, epoch)
    writer.add_scalar('loss_nse_val', val_loss, epoch)
    
    if val_loss < best_loss_model[1] - min_improvement:
        best_loss_model = (epoch, val_loss, model.state_dict())  # new best model
        load_data.pickle_model('ConvLSTM_simulationTraining', model, 'allStations', time_stamp)
    elif epoch > best_loss_model[0] + patience:
        print('Patience exhausted in epoch {}. Best val-loss was {}'.format(epoch, best_loss_model[1]))
        break

print('Using best model from epoch', str(best_loss_model[0]), 'which had loss', str(best_loss_model[1]))
model.load_state_dict(best_loss_model[2])
load_data.save_model_with_state('ConvLSTM_simulationTraining', best_loss_model[0], model, optimizer, time_stamp)

Epoch 0 mean train loss:	0.4203701615333557
Epoch 0 mean val loss:	0.2634117603302002
Saved model as ../pickle/models/ConvLSTM_simulationTraining_allStations_20190812-210928.pkl
Using best model from epoch 0 which had loss 0.2634117603302002
Saved model as ../pickle/models/ConvLSTM_simulationTraining_allStations_20190812-210928.pkl


In [24]:
logger.warning('predicting')
model.eval()

predictions = []
conv_hidden_states = None
for i, test_batch in enumerate(test_dataloader):
    if not stateful_lstm:
        conv_hidden_states = None
        
    geophysical_input = geophysical_batch[:test_batch['y_sim'].shape[0]]
    pred, conv_hidden_states = model(test_batch['x_conv'].to(device), geophysical_input, hidden_state=conv_hidden_states)
    predictions.append(pred.detach())
    
predictions = torch.cat(predictions).cpu()

if stateful_lstm:
    # reorder time series
    pred_indices = np.array(list(test_sampler.__iter__())).reshape(-1)
    predictions = predictions[pred_indices.argsort()]

2019-08-12 21:24:32,429 - 20190812-210928 - predicting


In [50]:
actuals = test_dataset.data_runoff.copy()
if len(actuals['date'].unique()) != len(predictions):
    print('Warning: length of prediction {} and actuals {} does not match.'.format(len(predictions), len(actuals['date'].unique())))

nse_dict, nse_sim_dict = {}, {}
mse_dict, mse_sim_dict = {}, {}
predictions_df = pd.DataFrame(columns=actuals.columns)
predictions_df['is_test_subbasin'] = False
predictions_df['is_val_subbasin'] = False
for subbasin in test_dataset.simulated_streamflow['subbasin'].unique():
    row, col = test_dataset.outlet_to_row_col[subbasin]
    
    station = None
    subbasin_sim = test_dataset.simulated_streamflow[test_dataset.simulated_streamflow['subbasin'] == subbasin].set_index('date')
    if subbasin in station_subbasins:
        station = subbasin_sim['StationID'].values[0]
        act = actuals[actuals['station'] == station].set_index('date')['runoff']
    if predictions.shape[0] != subbasin_sim.shape[0]:
        print('Warning: length of prediction {} and actuals {} does not match for subbasin {}. Ignoring excess actuals.'.format(len(predictions), len(subbasin_sim), subbasin))
        subbasin_sim = subbasin_sim.iloc[:predictions.shape[0]]
        if station is not None:
            act = act.iloc[:predictions.shape[0]]
    pred = pd.DataFrame({'runoff': predictions[:,row,col]}, index=subbasin_sim.index)
    pred['subbasin'] = subbasin
    pred['station'] = station
    pred['is_test_subbasin'] = subbasin in test_subbasins
    pred['is_val_subbasin'] = subbasin in val_subbasins
    predictions_df = predictions_df.append(pred.reset_index(), sort=True)
    nse_sim, mse_sim = evaluate.evaluate_daily('Sub{}'.format(subbasin), pred['runoff'], subbasin_sim['simulated_streamflow'], writer=writer)
    nse_sim_dict[subbasin] = nse_sim
    mse_sim_dict[subbasin] = mse_sim

    if station is not None:
        nse, mse = evaluate.evaluate_daily(station, pred['runoff'], act, writer=writer)
        nse_dict[subbasin] = nse
        mse_dict[subbasin] = mse
        print(station, subbasin, '\tNSE:', nse, '\tMSE:', mse, '(clipped to 0)')
    print(subbasin, '\tNSE sim:', nse_sim, '\tMSE sim:', mse_sim)

1 	NSE sim: -2.199903436086115 	MSE sim: 4451870.046758823
2 	NSE sim: -1.3574068228219613 	MSE sim: 245.96798408033803
3 	NSE sim: -1.4190235838105263 	MSE sim: 84.33877222041394
4 	NSE sim: -1.2910448127423937 	MSE sim: 49.764227231125425
5 	NSE sim: -1.3613226502707616 	MSE sim: 67.37450407202111
6 	NSE sim: -1.2948369019069674 	MSE sim: 13.831181967084712
7 	NSE sim: -1.2403838634012314 	MSE sim: 6.375985807725265
8 	NSE sim: -1.173582479999547 	MSE sim: 21.240025719732603
9 	NSE sim: -1.3892841364163528 	MSE sim: 10.38586589886836
10 	NSE sim: -1.3579036678697034 	MSE sim: 13.899923329365578
11 	NSE sim: -1.3226850803941623 	MSE sim: 33.01331525562825
12 	NSE sim: -1.3471414221468025 	MSE sim: 7.9037782244197095
13 	NSE sim: -1.2840839464382805 	MSE sim: 8.519430419928
14 	NSE sim: -1.608708099309601 	MSE sim: 686.9079900613818
15 	NSE sim: -1.5757775423296736 	MSE sim: 314.8054046858693
16 	NSE sim: -1.5954798922601543 	MSE sim: 36.54245098918974
17 	NSE sim: -1.553224071218672 	

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: invalid value encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


500 	NSE sim: -1.3026929626104953 	MSE sim: 48.60953842056637
501 	NSE sim: -1.5196797139929683 	MSE sim: 0.14849986867588713
502 	NSE sim: -1.1081653348959182 	MSE sim: 0.38138883416920094
503 	NSE sim: -1.046416508832428 	MSE sim: 0.09506771532302706
504 	NSE sim: -1.4254664061667008 	MSE sim: 0.02469419954094625
505 	NSE sim: -1.1411840383756244 	MSE sim: 10.279855973779354
506 	NSE sim: -1.1200660895225667 	MSE sim: 127.83839490359645
507 	NSE sim: -1.0269653468151838 	MSE sim: 0.045840822716370466
508 	NSE sim: -0.9123303593925118 	MSE sim: 9.487565285644626
509 	NSE sim: -1.094042686844594 	MSE sim: 212.17788579476164
510 	NSE sim: -0.8355801142233441 	MSE sim: 2.359129536164877
511 	NSE sim: -0.872880355851936 	MSE sim: 2.5744902646634125
512 	NSE sim: -1.1509433306457728 	MSE sim: 870.8463565736823
513 	NSE sim: -1.1378300639744476 	MSE sim: 818.7431629020939
514 	NSE sim: -0.7198215641994781 	MSE sim: 2.9401048836303016
515 	NSE sim: -0.7093146962485142 	MSE sim: 0.60006893542

/home/mgauch/miniconda3/envs/gwf/lib/python3.7/site-packages/hydroeval/objective_functions.py:31: RuntimeWarning: invalid value encountered in true_divide
  np.sum((evaluation - np.mean(evaluation)) ** 2, dtype=np.float64))


526 	NSE sim: -1.2512695908186373 	MSE sim: 0.0008151147284388762
527 	NSE sim: -0.8513177871372446 	MSE sim: 0.050788538199872536
528 	NSE sim: -0.5592543410150999 	MSE sim: 0.2153318247611047
529 	NSE sim: -0.641595036467435 	MSE sim: 0.031222182127349746
530 	NSE sim: -2.381396019491989 	MSE sim: 17.770255414557084
531 	NSE sim: -1.19692603968562 	MSE sim: 3.046535021955259
532 	NSE sim: -1.195312393100897 	MSE sim: 0.8470898228210056
533 	NSE sim: -1.0640631911346667 	MSE sim: 0.005355533056200794
534 	NSE sim: -0.5106822378774893 	MSE sim: 2.0479489760043665
535 	NSE sim: -1.9839969065659093 	MSE sim: 1.0364212911950679
536 	NSE sim: -2.3879640450427084 	MSE sim: 8.87855072071701
537 	NSE sim: -1.8354160163856057 	MSE sim: 5.136317403419322
538 	NSE sim: -1.2549160971813165 	MSE sim: 545.5309046030964
539 	NSE sim: -2.090483975873584 	MSE sim: 0.06129635276002474
540 	NSE sim: -0.9848446517485934 	MSE sim: 16407.08128978669
541 	NSE sim: -0.7081347440015489 	MSE sim: 1769.03195457

In [51]:
def print_nse_mse(name, nse_dict, mse_dict, subbasins):
    nses = list(nse_dict[s] for s in subbasins)
    mses = list(mse_dict[s] for s in subbasins)
    print(name, 'Median NSE (clipped to 0)', np.median(nses), '/ Min', np.min(nses), '/ Max', np.max(nses))
    print(name, 'Median MSE (clipped to 0)', np.median(mses), '/ Min', np.min(mses), '/ Max', np.max(mses))
    
    return np.median(nses)

print_nse_mse('Train sim', nse_sim_dict, mse_sim_dict, train_subbasins)
print_nse_mse('Val sim', nse_sim_dict, mse_sim_dict, val_subbasins)
nse_median_sim_test = print_nse_mse('Test sim', nse_sim_dict, mse_sim_dict, train_subbasins)
nse_median_stations_train_val = print_nse_mse('Stations (Train/Val)', nse_dict, mse_dict, list(s for s in station_subbasins if s not in test_subbasins))
nse_median_stations_test = print_nse_mse('Stations (Test)', nse_dict, mse_dict, list(s for s in station_subbasins if s in test_subbasins))
nse_median_stations = print_nse_mse('Stations (Train/Val/Test)', nse_dict, mse_dict, station_subbasins)

writer.add_scalar('nse_median_sim', nse_median_sim_test)
writer.add_scalar('nse_median_stations_test', nse_median_stations_test)
writer.add_scalar('nse_median_stations_all', nse_median_stations)

Train sim Median NSE (clipped to 0) -1.1348139726277142 / Min -2.77487097214355 / Max -0.4793329661695591
Train sim Median MSE (clipped to 0) 41.774158995956036 / Min 0.0008151147284388762 / Max 4467572.381746646
Val sim Median NSE (clipped to 0) -1.2100428640540348 / Min -2.7909682639207185 / Max -0.5227559903794339
Val sim Median MSE (clipped to 0) 61.75385241288536 / Min 0.021887717909923072 / Max 101353.26787278989
Test sim Median NSE (clipped to 0) -1.1348139726277142 / Min -2.77487097214355 / Max -0.4793329661695591
Test sim Median MSE (clipped to 0) 41.774158995956036 / Min 0.0008151147284388762 / Max 4467572.381746646
Stations (Train/Val) Median NSE (clipped to 0) -0.5473222983940333 / Min -3.1814051497781906 / Max -0.1970546100663817
Stations (Train/Val) Median MSE (clipped to 0) 322.9911407987569 / Min 19.653015302099394 / Max 150217.67265061528
Stations (Test) Median NSE (clipped to 0) -0.3239832416175117 / Min -1.0736174919684736 / Max -0.16661843148700117
Stations (Test) M

In [52]:
nse_dict

{676: -0.29384629963673725,
 677: -1.652597550040141,
 678: -0.23820604701538706,
 680: -0.32842831649941573,
 681: -0.2999791466956667,
 682: -0.24456839068939606,
 683: -0.4164159921113142,
 684: -0.3974007463340512,
 685: -0.5543173205743697,
 686: -0.5420122053863492,
 687: -1.0129355918887093,
 688: -0.4321788929005739,
 689: -0.6643421815376729,
 690: -0.6961722364047738,
 691: -2.1710831508606874,
 692: -1.1775894055874336,
 693: -1.3821456208772536,
 694: -0.2960670236978604,
 695: -0.2278622202136571,
 696: -0.4750116331042491,
 697: -0.2853377234310457,
 698: -0.3626287598039777,
 699: -0.1970546100663817,
 700: -0.24797658033928327,
 701: -0.39294938777671584,
 702: -0.6905369975551685,
 703: -2.291573222842996,
 704: -0.6473906285794009,
 705: -0.4887028054811109,
 706: -0.5139090198742158,
 707: -3.1814051497781906,
 709: -1.0736174919684736,
 710: -0.7516081518293949,
 712: -0.5526323914017175,
 713: -0.7493195786868887,
 714: -0.302561880478174,
 715: -0.3688997854151787

In [35]:
writer.close()

In [56]:
save_df = pd.merge(predictions_df.rename({'runoff': 'prediction'}, axis=1), 
                   train_dataset.simulated_streamflow, on=['date', 'subbasin'])
save_df = pd.merge(save_df, actuals.rename({'runoff': 'actual'}, axis=1), how='left', on=['date', 'station'])\
            [['date', 'subbasin', 'station', 'prediction', 'actual', 'simulated_streamflow', 'is_test_subbasin', 'is_val_subbasin']]
load_data.pickle_results('ConvLSTM_simulationTraining', save_df, time_stamp)

'ConvLSTM_simulationTraining_20190812-210928.pkl'

In [57]:
_ = print(train_subbasins), print(val_subbasins), print(test_subbasins)

[1, 3, 4, 5, 6, 10, 12, 13, 14, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 33, 34, 35, 36, 37, 39, 42, 43, 45, 46, 47, 48, 52, 54, 55, 57, 58, 59, 62, 63, 64, 66, 67, 70, 73, 74, 75, 78, 79, 81, 82, 84, 85, 87, 88, 89, 91, 92, 94, 95, 96, 98, 99, 100, 101, 102, 103, 105, 108, 109, 110, 111, 112, 113, 115, 116, 117, 118, 120, 121, 122, 124, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 138, 139, 140, 141, 142, 144, 145, 147, 148, 149, 150, 152, 153, 154, 155, 156, 159, 160, 161, 162, 164, 165, 166, 167, 168, 169, 174, 175, 177, 178, 179, 180, 181, 183, 184, 185, 187, 189, 190, 191, 192, 193, 194, 195, 196, 198, 199, 200, 201, 202, 203, 204, 206, 208, 209, 211, 212, 213, 214, 217, 218, 219, 220, 221, 223, 224, 225, 226, 227, 228, 229, 231, 233, 234, 235, 237, 238, 239, 241, 242, 243, 246, 249, 250, 251, 254, 255, 257, 260, 261, 262, 263, 265, 266, 270, 271, 273, 274, 276, 277, 278, 279, 280, 281, 282, 283, 285, 286, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 

In [58]:
datetime.now().strftime('%Y%m%d-%H%M%S')

'20190812-214437'